In [1]:
import imp
import os
import cv2
import pdb
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
# from torch.nn.functional import InterpolationMode
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import logging 
import datetime
import sys
import cv2
import matplotlib.pyplot as plt
import PIL

In [2]:
gloss_df_path = "data_validation/processed_gloss.csv"
gloss_df = pd.read_csv(gloss_df_path)
gloss_df.dropna(inplace=True)
gloss_df.replace(to_replace="ASHAG", value="AŞAĞI", inplace=True)
gloss_df['glossRange'] = gloss_df['glossEnd'] - gloss_df['glossStart']
# gloss_df.head()

In [3]:
classes = sorted(gloss_df.gloss.unique())
num_classes = len(classes)

In [4]:
class_to_idx = {classes[i]: i for i in range(len(classes))}
idx_to_class = {i: classes[i] for i in range(len(classes))}

In [5]:
import torchviz
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from torchvision import transforms

In [6]:
from pytorchvideo.data import labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute,   
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomAdjustSharpness,
    Resize,
    RandomHorizontalFlip
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [7]:
video_transform = Compose([
    ApplyTransformToKey(key="video",
    transform=Compose([
        UniformTemporalSubsample(50),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=256, max_size=512),
        CenterCropVideo(256),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [8]:
train_path = "/home/toghrul/SLR/data/binary-data/train"
val_path = "/home/toghrul/SLR/data/binary-data/val"
test_path = "/home/toghrul/SLR/data/binary-data/test"

In [9]:
bin_classes = []

for gloss in os.listdir(test_path):
    bin_classes.append(gloss)

In [10]:
bin_classes = sorted(bin_classes)
bin_class_to_idx = {bin_classes[i]: i for i in range(len(bin_classes))}
bin_idx_to_class = {i: bin_classes[i] for i in range(len(bin_classes))}
bin_idx_to_class

{0: 'MƏN', 1: 'VAR'}

In [11]:
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.metrics import classification_report
import torchmetrics

In [12]:
# video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)

In [13]:
# dataset = labeled_video_dataset(train_path, transform=video_transform, clip_sampler=make_clip_sampler('random', 2), decode_audio=False)

# loader = DataLoader(dataset, batch_size=8, num_workers=0, pin_memory=True)

In [14]:
# batch = next(iter(loader))

In [15]:
# batch.keys()

In [16]:
class VideoModel(LightningModule):
    def __init__(self, ):
        super(VideoModel, self).__init__()

        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(400, 1)
        # self.softmax = nn.Softmax(dim=0)
        # self.log_softmax = nn.LogSoftmax()
        self.sigmoid = nn.Sigmoid()
        
        self.clip_duraiton = 2
        self.lr = 1e-3
        self.batch_size = 8
        self.num_worker = 4
        self.num_steps_train = 0
        self.num_steps_val = 0

        # self.metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_classes)
        self.metric = torchmetrics.Accuracy()
        
        #loss
        # self.criterion = nn.BCEWithLogitsLoss()
        self.criterion = nn.BCELoss()

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.fc(x)
        # x = self.softmax(x)
        # x = self.log_softmax(x)
        x = self.sigmoid(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr = self.lr)
        # scheduler = ReduceLROnPlateau(opt, mode="min", factor=0.05, patience=2, min_lr=1e-6)
        scheduler = CosineAnnealingWarmRestarts(opt, T_0=10, T_mult=2, eta_min=1e-6, last_epoch=-1, verbose=True)
        return {'optimizer': opt,
                'lr_scheduler': scheduler, 
                "monitor": "train_loss"}

    def train_dataloader(self):
        dataset = labeled_video_dataset(train_path, clip_sampler=make_clip_sampler('random', self.clip_duraiton),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f">>> Training step No.{self.num_steps_train}:")
        print(f"Label: {label}\nPred: {out}")
        # print("Pred:", out)
        # print("GT:", label)
        # print(f"Pred:\n{out}")
        # print(f"Pred shape:\n{out.shape}")
        # print(f"Label:\n{label}")
        # print(f"Label shape:\n{label.shape}")
        # print(">>> INFO: Computing Training Loss")
        
        loss = self.criterion(out, label.float().unsqueeze(1))
        
        print(f"Loss: {loss}")
        self.num_steps_train += 1
        # print(">>> INFO: Training Loss Computed")
        # print(">>> INFO: Computing Training Metric")
        # metric = self.metric(out, label)
        
        # Below is for Accuracy
        metric = self.metric(out, label.to(torch.int64).unsqueeze(1))
        
        print(f"Accuracy: {metric}")

        values = {"loss": loss,
                "metric": metric.detach()}
        
        self.log_dict({"step_loss": loss,
                        "step_metric": metric.detach()})
        
        return values
        
        # return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        
        self.log('training_loss', loss)
        print(f">>> Epoch end loss: {loss}")
        self.log('training_metric', metric)
        

    def val_dataloader(self):
        dataset = labeled_video_dataset(val_path, clip_sampler=make_clip_sampler('random', self.clip_duraiton),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f">>> Validation step No.{self.num_steps_val}:")
        print(f"Label: {label}\nPred: {out}")
        # print(">>> INFO: Computing Val Loss")

        loss = self.criterion(out, label.float().unsqueeze(1))
        print()
        print(f"Loss: {loss}")
        self.num_steps_val += 1
        # print(">>> INFO: Val Loss Computed")
        # print(">>> INFO: Computing Val Metric")
        # metric = self.metric(out, label)
        # Below is for Accuracy
        
        metric = self.metric(out, label.to(torch.int64).unsqueeze(1))
                
        print(f"Accuracy: {metric}")
        

        return {"loss": loss,
                "metric": metric.detach()}
        
        # return {"loss": loss}

    def validation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = labeled_video_dataset(test_path, clip_sampler=make_clip_sampler('random', self.clip_duraiton),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self.forward(video)
        # metric = self.metric(out, label)

        return {"label": label,
                "pred": out.detach(),}

    def test_epoch_end(self, outputs):
        label=torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        # self.log('test_loss', loss)
        # self.log('test_metric', metric)
        
        # Below is for MultiLabelClassification
        # class_labels = label.argmax(axis=1)
        # class_pred = pred.argmax(axis=1)
        
        
        # Below is for BinaryClassification
        class_labels = label
        class_pred = np.where(pred > 0, 1, 0)
        
        print(f">> Label: {class_labels}")
        print(f">> Pred: {class_pred.squeeze()}")
              
        print(classification_report(class_labels, class_pred))
        
        return {"prediction": class_pred,
                "labels": class_labels}




In [17]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss',mode="min", dirpath="checkpoints", auto_insert_metric_name=True,
                                    verbose=True, save_last=True, save_top_k=2)
lr_monitor = LearningRateMonitor(logging_interval="epoch")


# Training

In [18]:
model = VideoModel()
seed_everything(0)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0


0

In [19]:
trainer = Trainer(max_epochs=100,
                accelerator="gpu", devices=-1,
                precision=16,
                # accumulate_grad_batches=2,
                enable_progress_bar=True,
                # num_sanity_val_steps=0,
                callbacks=[lr_monitor, checkpoint_callback],
                log_every_n_steps=5,
                limit_train_batches=25,
                limit_val_batches=10,
                limit_test_batches=10,)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
# trainer.fit(model, ckpt_path="/home/toghrul/SLR/sign-lang/checkpoints/last-v10.ckpt")
trainer.fit(model)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/toghrul/SLR/sign-lang/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type         | Params
---------------------------------------------
0 | video_model | EfficientX3d | 3.8 M 
1 | relu        | ReLU         | 0     
2 | fc          | Linear       | 401   
3 | sigmoid     | Sigmoid      | 0     
4 | metric      | Accuracy     | 0     
5 | criterion   | BCELoss      | 0     
---------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
7.589     Total estimated model params size (MB)


Epoch 00000: adjusting learning rate of group 0 to 1.0000e-03.


Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: Unable to find a valid cuDNN algorithm to run convolution

In [20]:
# model = VideoModel.load_from_checkpoint(
#     checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/last-v4.ckpt",
#     hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_45/hparams.yaml",
#     map_location=None,
# )
# model = model.cuda()

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [22]:
model_path = "../models/eff3d_bin.pt"
# torch.save(model.state_dict(), model_path)

# Testing

In [47]:
test_res = trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

>> Label: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0]
>> Pred: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        68
           1       1.00      0.92      0.96        12

    accuracy                           0.99        80
   macro avg       0.99      0.96      0.97        80
weighted avg       0.99      0.99      0.99        80



# Inferencing

In [21]:
from pytorchvideo.data.encoded_video import EncodedVideo

In [60]:
model_path = "../models/eff3d_bin.pt"
video_path = "../data/binary-data/test/MƏN/2022-04-26 12-58-07.mp4"
video = EncodedVideo.from_path(video_path)

In [61]:
video_data = video.get_clip(0, 1)
print(video_data['video'].shape)
video_data = video_transform(video_data)

inputs = video_data["video"].cuda()
inputs = inputs.unsqueeze(0)
inputs.shape

torch.Size([3, 15, 960, 1280])


torch.Size([1, 3, 25, 256, 256])

In [62]:
video_data['video'].shape

torch.Size([3, 25, 256, 256])

In [63]:
preds = model(inputs).detach().cpu().numpy()
preds = np.where(preds > 0, 1, 0)
bin_idx_to_class[preds[0, 0]]

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [34]:
# model_path = "../models/eff3d_bin.pt"
video_path = "../data/binary-data/test/VAR/2022-05-24 15-56-37.mp4"

In [66]:
def word_level_prediction(path_to_model, path_to_video):
    
    model = VideoModel()
    # model.load_state_dict(torch.load(path_to_model))
    
    model = VideoModel.load_from_checkpoint(
    checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/epoch=32-step=825.ckpt",
    # hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_49/hparams.yaml",
    map_location=None,
    )
    
    model = model.cuda()
    
    video = EncodedVideo.from_path(path_to_video)

    video_data = video.get_clip(0, 2)
    video_data = video_transform(video_data)

    inputs = video_data["video"].cuda()
    inputs = inputs.unsqueeze(0)
    
    preds = model(inputs).detach().cpu().numpy()
    # preds = np.where(preds > 0, 1, 0)
    
    return preds

In [71]:
word_level_prediction(model_path, video_path)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


array([[-6.538341]], dtype=float32)

# Validation

In [ ]:
# val_res = trainer.validate(model)